**Baseline classifier model. Op basis van TF-IDF Logistische regressie. Moet class imbalance nog aanpakken.**

In [5]:
""""
wat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"
daarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"
"""


'"\nwat gebeurd is: eerst GridSearchCV gerund en daarna gezien dat deze configuratie met l1 dus de beste is"\ndaarna dus de l1 configuratie gebruikt en de beste parameters nog aan het zoeken"\n'

In [6]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd


import os

In [7]:
# ✅ 1. Load & Preprocess Data
script_dir = os.path.dirname(os.getcwd())# Ga één map omhoog om 'baseline' te verwijderen en ga naar 'Data'
project_root = os.path.dirname(script_dir)  # Dit verwijdert 'baseline' van het script_dir
data_folder = os.path.join(project_root, "Data")

# 1. Dataset inladen
file_path = os.path.join(data_folder, r"C:\Users\jefva\Documents\Master\Thesis_s2\Code\Identifier\Trainig_data.xlsx")
df = pd.read_excel(file_path)

#visualize the data
print(df.head())
print(df.info())

                                            question  label
0  Hoeveel personen waren er voor de Vlaamse over...      1
1  Hoeveel dagen/uren is die delegatie er geweest...      0
2                             Hoe verklaart hij dat?      0
3  Hoeveel bedroeg de totale factuur voor de Vlaa...      0
4  Verder sprak ik de Catalaanse minister van Fin...      0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  2386 non-null   object
 1   label     2386 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.4+ KB
None


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import nltk

# === Setup
nltk.download("stopwords")
from nltk.corpus import stopwords
dutch_stopwords = stopwords.words("dutch")



# === Split the data with row IDs tracked
X_train, X_temp, y_train, y_temp = train_test_split(
    df["question"].tolist(),
    df["label"].tolist(),
    test_size=0.3,
    random_state=42,
    stratify=df["label"]
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

# === Save test set with only IDs and theme IDs
# pd.DataFrame({
#     "clean_text": X_test,
#     "label": y_test
# }).to_excel("Test_data_HeldOut_15percentlogreg.xlsx", index=False)

# === TF-IDF vectorization
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=10000,
    stop_words=dutch_stopwords
)

X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jefva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
import time
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
import pandas as pd

# === 4. Logistic Regression + GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'penalty': ['l2'],
    'solver': ['saga'],
    'class_weight': ['balanced']
}
#these can also be tried, evaded for now due to time 
#  {
#     'C': [0.1, 1, 10, 100],
#     'penalty': ['elasticnet'],
#     'solver': ['saga'],
#     'class_weight': ['balanced'],
#     'l1_ratio': [0.3, 0.5, 0.7]  # reduce range for speed
# }


#     'C': [0.1, 1, 10],
#     'penalty': ['l1'],
#     'solver': ['saga'],
#     'class_weight': ['balanced']
# }
grid = GridSearchCV(
    LogisticRegression(max_iter=1000),
    param_grid,
    cv=5,
    scoring="f1_weighted",
    n_jobs=-1,
    verbose=2
)

print("GridSearchCV wordt uitgevoerd... 🚀")
start_time = time.time()
grid.fit(X_train_vec, y_train)
end_time = time.time()
print(f"⏱️ Trainingstijd: {end_time - start_time:.2f} seconden")

# === 5. Predict on validation set
best_model = grid.best_estimator_
y_pred = best_model.predict(X_val_vec)

print(f"Beste parameters: {grid.best_params_}")
print("=== Evaluatie op validatieset ===")
print(classification_report(y_val, y_pred, zero_division=0))

# === 6. Summary table
accuracy = accuracy_score(y_val, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_val, y_pred, average="weighted", zero_division=1)

baseline_results = pd.DataFrame({
    "Metric": ["Accuracy", "Precision", "Recall", "F1"],
    "Score": [accuracy, precision, recall, f1]
})
print(baseline_results)

# === 7. Save validation predictions to Excel (with IDs)
df_val_predictions = pd.DataFrame({
    "True Label": y_val,
    "Predicted Label": y_pred
})
df_val_predictions.to_excel("logreg_val_predictions.xlsx", index=False)
print("📄 Validatievoorspellingen opgeslagen in 'logreg_val_predictions.xlsx'")

# === 8. Predict on test set
y_test_pred = best_model.predict(X_test_vec)

# === 9. Classification report on test set
print("=== Evaluatie op test set ===")
print(classification_report(y_test, y_test_pred, zero_division=0))

# === 10. Save test predictions to Excel (with IDs)
df_test_predictions = pd.DataFrame({
    "clean_text": X_test,
    "True Label": y_test,
    "Predicted Label": y_test_pred
})
df_test_predictions.to_excel("logreg_test_predictions.xlsx", index=False)
print("📄 Testvoorspellingen opgeslagen in 'logreg_test_predictions.xlsx'")


GridSearchCV wordt uitgevoerd... 🚀
Fitting 5 folds for each of 5 candidates, totalling 25 fits
⏱️ Trainingstijd: 0.51 seconden
Beste parameters: {'C': 10, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'saga'}
=== Evaluatie op validatieset ===
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       282
           1       0.62      0.63      0.62        76

    accuracy                           0.84       358
   macro avg       0.76      0.76      0.76       358
weighted avg       0.84      0.84      0.84       358

      Metric     Score
0   Accuracy  0.837989
1  Precision  0.839579
2     Recall  0.837989
3         F1  0.838753
📄 Validatievoorspellingen opgeslagen in 'logreg_val_predictions.xlsx'
=== Evaluatie op test set ===
              precision    recall  f1-score   support

           0       0.92      0.89      0.90       283
           1       0.63      0.71      0.67        75

    accuracy                           0.8